In [17]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [19]:
from bs4 import BeautifulSoup
import bs4
from datetime import datetime

In [20]:
pip install mailslurp-client


Note: you may need to restart the kernel to use updated packages.


In [21]:
import mailslurp_client

configuration = mailslurp_client.Configuration()
configuration.api_key['x-api-key'] = ""

In [22]:
import pprint
import mailslurp_client
from mailslurp_client.rest import ApiException

In [23]:
pp = pprint.PrettyPrinter(indent=4)

In [24]:
# email_id = '9ac55027-b2f1-43af-bf8e-6c36a7888294'
email_id= '67fd1c5b-53ab-4840-804d-95ab2d8df5a1'#'d10fa4d3-880b-46b1-9c30-2ba829c19d4e'#'8a079ecb-9d84-4045-801e-350a3ed8df5f'

In [25]:
with mailslurp_client.ApiClient(configuration) as api_client:
    api_instance = mailslurp_client.EmailControllerApi(api_client)
    decode = False
    try:
        api_response = api_instance.get_email_html(email_id, decode=decode)
#         pp.pprint(api_response)
    except ApiException as e:
        print("Exception when calling EmailControllerApi->get_email_html: %s\n" % e)

In [26]:
pip install lxml


Note: you may need to restart the kernel to use updated packages.


In [27]:
soup = BeautifulSoup(api_response, 'lxml')

In [28]:
soup.find_all('h4')

[<h4>********* INDEX ***********</h4>,
 <h4>Biotech and Healthcare</h4>,
 <h4>Business and Finance </h4>,
 <h4>Education </h4>,
 <h4>General</h4>,
 <h4>Lifestyle and Fitness</h4>,
 <h4>Public Policy and Government</h4>,
 <h4>****************************</h4>,
 <h4><strong>Biotech and Healthcare</strong></h4>,
 <h4><strong>Business and Finance </strong></h4>,
 <h4><strong>Education </strong></h4>,
 <h4><strong>General</strong></h4>,
 <h4><strong>Lifestyle and Fitness</strong></h4>,
 <h4><strong>Public Policy and Government</strong></h4>]

In [29]:
categories = soup.find_all('h4')

In [30]:
all_links = {}
for h in categories:
    a = h.find_all('a')
    all_links[h.text] = a
# html = u""
# for tag in soup.find("h4").next_siblings:
#     if tag.name == "h4":
#         break
#     else:
#         html += str(tag)

In [31]:
categories

[<h4>********* INDEX ***********</h4>,
 <h4>Biotech and Healthcare</h4>,
 <h4>Business and Finance </h4>,
 <h4>Education </h4>,
 <h4>General</h4>,
 <h4>Lifestyle and Fitness</h4>,
 <h4>Public Policy and Government</h4>,
 <h4>****************************</h4>,
 <h4><strong>Biotech and Healthcare</strong></h4>,
 <h4><strong>Business and Finance </strong></h4>,
 <h4><strong>Education </strong></h4>,
 <h4><strong>General</strong></h4>,
 <h4><strong>Lifestyle and Fitness</strong></h4>,
 <h4><strong>Public Policy and Government</strong></h4>]

In [32]:
for e in soup.findAll('br'):
    e.extract()

In [33]:
reporters_in_category = {}
for category in categories:
    all_tags = []
    for tag in category.next_siblings:
        if tag.name == "h4":
            break
        else:
            all_tags.append(tag)
    reporters_in_category[category.text] = all_tags

In [34]:
reporters_in_category


  '\r\nDeadline: 7:00 PM EST - 24 May\r\n',
  '\n',
  <p>
  Query: 
  
   Hello, My name is Julie and I am the owner of Emmy's Deals. Weare putting together different gift guides this year. We arelooking for amazing products to feature in our gift guides. Weare looking from adult items- baby items. We are open to manydifferent ideas. We would love to feature your product in one ofour guides.
  </p>,
  '\n',
  <p><a href="#top">Back to Top</a> <a href="#General">Back to Category Index</a></p>,
  '\n',
  <p>-----------------------------------</p>,
  '\r\n\r\n13)',
  <a id="asuc" name="asuc"></a>,
  ' Summary: Anniversary Gifts For Him On Little Helpers In Life\r\n',
  '\n',
  <p>Name: Amanda Osburn
  Little Helpers In Life
  </p>,
  'Category: General\r\n',
  '\r\nEmail: ',
  <a href="mailto:query-asuc@helpareporter.net">query-asuc@helpareporter.net</a>,
  '\n',
  '\r\nMedia Outlet: Little Helpers In Life',
  '\r\nDeadline: 7:00 PM EST - 24 May\r\n',
  '\n',
  <p>
  Query: 
  
  Hi, my 

In [35]:
stories = []
story = None
final_category = None
for category in reporters_in_category:
    if "***" not in category:
        final_category = category
        for each_block in reporters_in_category[category]:
#             if isinstance(each_block, bs4.element.Tag):
#                 print(":::::::::::::::::::::::::::::::")
#                 print(each_block)
            if isinstance(each_block, bs4.element.NavigableString):
                if "Summary" in each_block.string:
                    summary = each_block.string.split("Summary: ")[1]
                    if story is not None:
                        story["category"] = category
                        stories.append(story)
                        story = None
                    story = {"summary": summary}
                if "Media Outlet" in each_block.string:
                    media_outlet = each_block.string.split("Media Outlet: ")[1]
                    story["media_outlet"] = media_outlet   
                if "Deadline" in each_block.string:
                    deadline = each_block.string.split("Deadline: ")[1].split("-")
                    deadline_date = deadline[1]
                    deadline_time = deadline[0]
                    story["deadline_date"] = deadline_date
                    story["deadline_time"] = deadline_time
                if "12051 Indian Creek Ct., Beltsville, MD 20705, USA" in each_block.string:
                    break
            if isinstance(each_block, bs4.element.Tag):
                if each_block.name == 'p':
                    if "Name: " in each_block.text:
                        story["name"] = each_block.text.split("Name: ")[1] 
                    if "Query specifics:" in each_block.text:
                        story["query_specifics"] = each_block.text.split("Query specifics: ")[1] 
                    if "Query:" in each_block.text:
                        story["query"] = each_block.text.split("Query: ")[1] 
                    if "Requirements" in each_block.text:
                        story["requirements"] = each_block.text.split("Requirements: ")[1]
                if each_block.name == 'a':
                    if "target" in each_block.attrs:
                        story["email"] = each_block.attrs['href']
story["category"] = final_category
if story is not None:
    stories.append(story)

In [36]:
len(stories)

29

In [37]:
pip install airtable-python-wrapper

Note: you may need to restart the kernel to use updated packages.


In [38]:
from airtable import Airtable
import os
os.environ["AIRTABLE_API_KEY"] = ""
airtable = Airtable('', '')

In [39]:
airtable.get_all()

 so thediscussion needs to be in-depth.\n",
   'deadline_date': ' 23 May\r\n',
   'deadline_time': '7:00 PM EST '},
  'createdTime': '2020-05-22T00:12:47.000Z'},
 {'id': 'recs5dsTqN9ivkTRZ',
  'fields': {'id': 354,
   'category': 'Lifestyle and Fitness',
   'summary': 'Bikes / Bicycles\r\n',
   'media_outlet': 'Beauty Brite',
   'name': 'Stephanie F\r\nBeauty Brite\r\n',
   'query': "\r\n\r\n We are in search of products to review! We are stuck at home.We're looking for bikes, bicycles, bikes with baskets, etc. Wewill feature/cover/review your products on the blog and share onsocial media. Are you interested in getting your product infront of thousands of blog readers and social media followers?We are growing every day! Beauty Brite complies with FTCguidelines.\n",
   'requirements': '\r\n\r\nMust be fitness/health/wellness related items. Must be aproduct, not a service.\n',
   'deadline_date': ' 21 May\r\n',
   'deadline_time': '10:00 PM EST '},
  'createdTime': '2020-05-22T00:12:49.0

In [40]:
for story in stories:
    airtable.insert(story)
print("Done")

Done
